In [7]:
from encoder_decoder_correlations import EncoderDecoderComparison
import torch
from lfxai.models.images import ClassifierMnist, EncoderMnist
import os
import copy

from captum.attr import GradientShap, IntegratedGradients, Saliency

In [2]:
# Define globals
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'

In [3]:
#### Model loading ######
# Specification
dim_latent = 4
name = "TestClassifier"
classifier_state_dict_path = os.path.join('..', 'TrainedModels', 'MNIST', 'Classifier_run0.pt') # TODO Need to loop over runs

# Load
encoder = EncoderMnist(dim_latent)

classifier = ClassifierMnist(encoder, dim_latent, name)
classifier.load_state_dict(torch.load(classifier_state_dict_path), strict=True)

encoder = copy.deepcopy(classifier.encoder) # Copies the params to the encoder variable

print("Classifier Loaded")

Classifier Loaded


## Gradshap

In [4]:
gradshap_full_model = GradientShap(classifier)
gradshap_encoder = GradientShap(encoder)

gradient_shap_experiment = EncoderDecoderComparison(
    model_name="",
    full_model=classifier,
    encoder = encoder,
    full_model_attributer=gradshap_full_model,
    encoder_attributer=gradshap_encoder,
    dataset='MNIST',
    device = DEVICE,
)

Calculating Attributions
attributions calculated


In [5]:
gradient_shap_image_pearsons = gradient_shap_experiment.get_all_image_pearsons()
gradient_shap_image_pearsons.mean() # Delete - just checking that this looks approximately correct

## Integrated Gradients

In [8]:
ig_full_model = IntegratedGradients(classifier)
ig_encoder = IntegratedGradients(encoder)

ig_experiment = EncoderDecoderComparison(
    model_name="",
    full_model=classifier,
    encoder = encoder,
    full_model_attributer=ig_full_model,
    encoder_attributer=ig_encoder,
    dataset='MNIST',
    device = DEVICE,
)

Calculating Attributions
attributions calculated


In [16]:
ig_pearsons = ig_experiment.get_all_image_pearsons()
ig_pearsons.mean()**2 # Delete - just checking that this looks approximately correct

0.2243376760550648

## Saliency

In [20]:
sal_full_model = Saliency(classifier)
sal_encoder = Saliency(encoder)

sal_experiment = EncoderDecoderComparison(
    model_name="",
    full_model=classifier,
    encoder = encoder,
    full_model_attributer=sal_full_model,
    encoder_attributer=sal_encoder,
    dataset='MNIST',
    device = DEVICE,
)

Calculating Attributions


C:\Users\niksm\Documents\CodeForUni\FACT\venv\lib\site-packages\captum\_utils\gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


attributions calculated


In [23]:
sal_pearsons = sal_experiment.get_all_image_pearsons()
sal_pearsons.mean()

0.13540589165468833